In [54]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import sys
import re
import json


print("Neccessary library installed")

Neccessary library installed


In [55]:
# New file path for the uploaded .xlsx file
xls_path = "C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/CanvasFiles/Raw Data/Scats Data October 2006.xls"

# Attempt to read the .xlsx file using openpyxl engine
df = pd.read_excel(xls_path, sheet_name="Data")

new_header = df.iloc[0]  # get the second row
df = df[1:]  # drop the first two rows (0 and 1)
df.columns = new_header  # set the second row as header

# Save it as a CSV file
csv_path = "C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/Scats Data October 2006.csv"
df.to_csv(csv_path, index=False)
print("The Data file has extracted and converted in to CSV\n")

print("Time header is remove for easy file read and process\n")
print("Vheicle total count is happen every 15mins, total of 96 count of a day:\n - V0 represent the time (0:00am)\n - V95 reprensent the time (11:59pm)")



The Data file has extracted and converted in to CSV

Time header is remove for easy file read and process

Vheicle total count is happen every 15mins, total of 96 count of a day:
 - V0 represent the time (0:00am)
 - V95 reprensent the time (11:59pm)


In [56]:
# Basic information of the Data
print("Display basic information of the Data")

df = pd.read_csv("C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/Scats Data October 2006.csv")
print("\nTraffic data header:")
print(df.head())

print("\nTraffic data summary (info):")
print(df.info())

print("\nMissing values per column in traffic data:")
print(df.isnull().sum())


print("Display basic information of the Data")

Display basic information of the Data

Traffic data header:
   SCATS Number                         Location CD_MELWAY  NB_LATITUDE  \
0           970  WARRIGAL_RD N of HIGH STREET_RD   060 G10    -37.86703   
1           970  WARRIGAL_RD N of HIGH STREET_RD   060 G10    -37.86703   
2           970  WARRIGAL_RD N of HIGH STREET_RD   060 G10    -37.86703   
3           970  WARRIGAL_RD N of HIGH STREET_RD   060 G10    -37.86703   
4           970  WARRIGAL_RD N of HIGH STREET_RD   060 G10    -37.86703   

   NB_LONGITUDE  HF VicRoads Internal  VR Internal Stat  VR Internal Loc  \
0     145.09159                   249               182                1   
1     145.09159                   249               182                1   
2     145.09159                   249               182                1   
3     145.09159                   249               182                1   
4     145.09159                   249               182                1   

   NB_TYPE_SURVEY               

In [57]:
zero_coord_rows = df[(df['NB_LATITUDE'] == 0) | (df['NB_LONGITUDE'] == 0)]
print(f"Number of rows with Zero Latitude or Longitude: {len(zero_coord_rows)}")
if not zero_coord_rows.empty:
    print("\nSCATS Numbers corresponding to rows with Zero Coordinates:")
    # Using unique() ensures we list each affected SCATS site only once, sorted() makes it tidier
    print(sorted(zero_coord_rows['SCATS Number'].unique()))
    print("Number of row have zero Latitude or Longitude: ",len(zero_coord_rows))
    # Optional: display some of these rows
    # print("\nExample rows with Zero Coordinates:")
    # print(zero_coord_rows.head())

    # (optional) display all the row that has altitude = 0 and longitude = 0
    #for idx, row in zero_coord_rows.iterrows():
        #print(f"SCATS Number: {row['SCATS Number']} | Location: {row['Location']} | Latitude: {row['NB_LATITUDE']} | Longitude: {row['NB_LONGITUDE']}")

else:
    print("No rows found with zero coordinates.")


# By searching through the internet, the Latitude and longitude will be -37.8240/145.0436
# Replace all the row's altitude and longitude of the the following row
mask = (df['SCATS Number'] == 4266) & (df['Location'] == "AUBURN_RD N of BURWOOD_RD")

# Update the latitude and longitude for these rows
df.loc[mask, 'NB_LATITUDE'] = -37.8240
df.loc[mask, 'NB_LONGITUDE'] = 145.0436
print(f"Updated {mask.sum()} rows with new latitude and longitude.")

# Remove the rows that is serve to count number of pedestrians, SCAT number 4335
mask = df['SCATS Number'] == 4335
df = df[~mask]
print(f"Removed {mask.sum()} rows with SCATS Number 4335.")


df.to_csv("C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/Scats Data October 2006.csv", index = False)
print("CSV file has been saved after removing the rows with zero coordinates and SCATS Number 4335.\n")

Number of rows with Zero Latitude or Longitude: 31

SCATS Numbers corresponding to rows with Zero Coordinates:
[4266]
Number of row have zero Latitude or Longitude:  31
Updated 31 rows with new latitude and longitude.
Removed 62 rows with SCATS Number 4335.
CSV file has been saved after removing the rows with zero coordinates and SCATS Number 4335.



In [58]:
# Purpose: To find the SCATS Number and Location with less than 31 dates of data
# Group by 'SCAT_Number' and 'Location' and count number of rows for each combination
grouped_counts = df.groupby(['SCATS Number', 'Location']).size().reset_index(name='Row_Count')

small_groups = grouped_counts[grouped_counts['Row_Count'] < 31]

    # For each of these groups, display their information
for _, row in small_groups.iterrows():
    scat_num = row['SCATS Number']
    location = row['Location']
    count = row['Row_Count']
    print(f"SCATS Number: {scat_num}, Location: {location}, Rows: {count}")



        

SCATS Number: 970, Location: HIGH STREET_RD W of WARRIGAL_RD, Rows: 30
SCATS Number: 2000, Location: BURWOOD_HWY E of WARRIGAL_RD, Rows: 29
SCATS Number: 2000, Location: TOORAK_RD W of WARRIGAL_RD, Rows: 29
SCATS Number: 2000, Location: WARRIGAL_RD N of TOORAK_RD, Rows: 28
SCATS Number: 2000, Location: WARRIGAL_RD S of BURWOOD_HWY, Rows: 28
SCATS Number: 2820, Location: PRINCESS_ST S of CHANDLER_HWY, Rows: 30
SCATS Number: 3001, Location: CHURCH_ST SW of BARKERS_RD, Rows: 2
SCATS Number: 3002, Location: BARKERS_RD E of DENMARK_ST, Rows: 26
SCATS Number: 3002, Location: BARKERS_RD W of DENMARK_ST, Rows: 26
SCATS Number: 3002, Location: DENMARK_ST N of BARKERS_RD, Rows: 26
SCATS Number: 3002, Location: POWER_ST S of BARKERS_RD, Rows: 26
SCATS Number: 3127, Location: BALWYN_RD N of CANTERBURY_RD, Rows: 26
SCATS Number: 3127, Location: CANTERBURY_RD E of BALWYN_RD, Rows: 26
SCATS Number: 3127, Location: CANTERBURY_RD W of BALWYN_RD, Rows: 26
SCATS Number: 3682, Location: RIVERSDALE_RD E OF

In [59]:
import pandas as pd
import numpy as np


df = pd.read_csv("C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/Scats Data October 2006.csv")
# Fix column names (in case there are extra spaces or invisible characters)
df.columns = df.columns.str.strip()

# Convert date column to datetime
df['Date'] = pd.to_datetime(df['Date'], format='mixed', dayfirst=True, errors='coerce').dt.normalize()


# Remove any rows with invalid dates
df = df.dropna(subset=['Date'])

# ----- STEP 2: Prepare -----
vehicle_cols = [f'V{i:02}' for i in range(96)]
oct_dates = pd.date_range(start='2006-10-01', end='2006-10-31').normalize()

filled_groups = []

# ----- STEP 3: Fill Missing Dates -----
for (scats, loc), group in df.groupby(['SCATS Number', 'Location']):
    group = group.set_index('Date').sort_index()
    group = group[~group.index.duplicated(keep='first')]
    existing_dates = group.index
    missing_dates = oct_dates.difference(existing_dates)

    if len(missing_dates) == 0 and len(existing_dates) == 31:
        # No missing data, keep original
        filled_groups.append(group.reset_index())
        continue

    # Calculate mean vehicle profile
    avg_row = group[vehicle_cols].mean().round().astype(int)

    # Use metadata from the first valid row
    meta = group.iloc[0][['SCATS Number', 'Location', 'CD_MELWAY', 'NB_LATITUDE', 'NB_LONGITUDE', 'HF VicRoads Internal']]

    # Create missing rows
    for date in missing_dates:
        new_row = avg_row.copy()
        for col in meta.index:
            new_row[col] = meta[col]
        new_row['Date'] = date
        filled_groups.append(pd.DataFrame([new_row]))

    # Add original existing data
    existing_group = group.loc[~group.index.isin(missing_dates)].reset_index()
    filled_groups.append(existing_group)


# ----- STEP 4: Combine, format, save -----
if not filled_groups:
    raise ValueError("No groups found to process. Check your input data.")

df_filled = pd.concat(filled_groups, ignore_index=True)

# Format date
df_filled['Date'] = pd.to_datetime(df_filled['Date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Sort and reorder columns
ordered_cols = ['SCATS Number', 'Location', 'CD_MELWAY', 'NB_LATITUDE', 'NB_LONGITUDE', 'HF VicRoads Internal', 'Date'] + vehicle_cols
df_filled = df_filled[ordered_cols].sort_values(by=['SCATS Number', 'Location', 'Date'])

# Save output
df_filled.to_csv("C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/Oversampling Scats Data October 2006.csv", index=False)
print("Done. Dataset oversampled with missing dates filled and properly formatted.")


Done. Dataset oversampled with missing dates filled and properly formatted.


In [60]:
# Purpose: To re-check the SCATS Number and Location with less than 31 dates of data

df = pd.read_csv("C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/Oversampling Scats Data October 2006.csv")
# Group by 'SCAT_Number' and 'Location' and count number of rows for each combination
grouped_counts = df.groupby(['SCATS Number', 'Location']).size().reset_index(name='Row_Count')

small_groups = grouped_counts[grouped_counts['Row_Count'] < 31]

    # For each of these groups, display their information
if small_groups.empty:
    print("All SCATS/Location combinations have 31 days of data.")
else:
    print("SCATS/Locations with fewer than 31 days:")
    for _, row in small_groups.iterrows():
        scat_num = row['SCATS Number']
        location = row['Location']
        count = row['Row_Count']
        print(f"SCATS Number: {scat_num}, Location: {location}, Rows: {count}")

        

All SCATS/Location combinations have 31 days of data.


In [61]:
unique_location_counts = df.groupby('SCATS Number')['Location'].nunique()
print(unique_location_counts)

unique_scat_numbers = df['SCATS Number'].unique()
print("\nAll unique SCATS Numbers in the data:")
for scat_num in unique_scat_numbers:
    print(scat_num)

SCATS Number
970     4
2000    4
2200    4
2820    2
2825    1
2827    4
2846    4
3001    4
3002    4
3120    4
3122    3
3126    3
3127    3
3180    3
3662    4
3682    4
3685    3
3804    4
3812    4
4030    3
4032    4
4034    4
4035    4
4040    6
4043    4
4051    3
4057    4
4063    4
4262    1
4263    4
4264    4
4266    4
4270    4
4272    3
4273    2
4321    4
4324    3
4812    3
4821    4
Name: Location, dtype: int64

All unique SCATS Numbers in the data:
970
2000
2200
2820
2825
2827
2846
3001
3002
3120
3122
3126
3127
3180
3662
3682
3685
3804
3812
4030
4032
4034
4035
4040
4043
4051
4057
4063
4262
4263
4264
4266
4270
4272
4273
4321
4324
4812
4821


In [62]:
# Convert wide format SCATS data to long format
# Present in the wide format, each row represents a SCATS site, with columns for each 15-minute interval (V00, V01, ..., V95).

import re
import pandas as pd

# 2. Your fixed metadata columns
fixed_cols = [
    "SCATS Number",
    "Location",
    "NB_LATITUDE",
    "NB_LONGITUDE",
    "Date"
]

# 3. Identify the V-columns ("V00" … "V95")
v_cols = [c for c in df.columns if re.match(r"^V\d{2}$", c)]

# 4. Melt into long form
df_long = df.melt(
    id_vars=fixed_cols,
    value_vars=v_cols,
    var_name="Quarter",    # holds "V00", "V01", …
    value_name="Count"     # the vehicle count
)

# 5. Extract the numeric quarter index and cast to int
df_long["QtrNum"] = (
    df_long["Quarter"]
      .str.extract(r"V(\d{1,2})$")[0]
      .astype(int)
)

# 6. Build the actual timestamp at *end* of that 15-min block:
#      V00 → end at 00:15, V01 → end at 00:30, … V95 → end at 24:00
df_long["DateTime"] = (
    pd.to_datetime(df_long["Date"]) +
    pd.to_timedelta((df_long["QtrNum"]) * 15, unit="m")
)

# 7. Drop the helper columns if you like
df_long = df_long.drop(columns=["Quarter", "QtrNum"])

# 8. Sort by SCATS Number, DateTime, then Location
# 8. Sort by SCATS→Location→DateTime, so each location’s times are contiguous & ordered
df_long = df_long.sort_values(
    by=["SCATS Number", "Location", "DateTime"]
)


# 9. Reorder columns exactly as you asked
df_long = df_long[
    [
        "SCATS Number",
        "Location",
        "NB_LATITUDE",
        "NB_LONGITUDE",
        "DateTime",   # now has date+time
        "Count"
    ]
]

# 10. Write out
df_long.to_csv("C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/Sorted Scats Data October 2006.csv", index=False)

print("Done — results in Sorted Scats Data October 2006.csv")


Done — results in Sorted Scats Data October 2006.csv


In [63]:
# split_scats.py
# Split a large CSV file into smaller files, one per SCATS Number & Location

import os
import re
import pandas as pd

# ←– EDIT THESE TWO PATHS –→
df = pd.read_csv("C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/Sorted Scats Data October 2006.csv")

def sanitize_filename(s: str) -> str:
    """
    Turn an arbitrary string into a safe filename:
      - lowercase, replace spaces & slashes with underscores
      - drop other non-alphanumeric/_/-
    """
    s = s.lower().replace('/', '_').replace(' ', '_')
    return re.sub(r'[^a-z0-9_\-]', '', s)

def split_scats(input_csv: str, outdir: str) -> None:
    # 1) Load full CSV, parse DateTime
    df = pd.read_csv(input_csv, parse_dates=['DateTime'])
    
    # 2) Ensure output directory exists
    os.makedirs(outdir, exist_ok=True)
    
    # 3) Group by SCATS Number & Location
    grouped = df.groupby(['SCATS Number', 'Location'])
    
    # 4) For each group, write a separate CSV
    for (scats_num, location), group in grouped:
        # a) Build a filesystem-safe filename
        scats_str = str(scats_num)
        loc_str   = sanitize_filename(location)
        fname     = f"scats_{scats_str}__{loc_str}.csv"
        outpath   = os.path.join(outdir, fname)
        
        # b) Sort by DateTime (optional but recommended)
        group = group.sort_values('DateTime')
        
        # c) Select and write the desired columns
        cols_to_write = [
            'SCATS Number',
            'Location',
            'NB_LATITUDE',
            'NB_LONGITUDE',
            'DateTime',
            'Count'
        ]
        # If any column is missing, this will raise a KeyError—catch if you need to debug:
        try:
            subset = group[cols_to_write]
        except KeyError as e:
            missing = list(set(cols_to_write) - set(group.columns))
            raise KeyError(f"Missing expected columns in master CSV: {missing}") from e
        
        subset.to_csv(outpath, index=False)
        print(f"Saved {len(group)} rows → {outpath}")

if __name__ == '__main__':
    split_scats(
        "C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/Sorted Scats Data October 2006.csv",
        "C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/per_site_csvs"
    )


Saved 2976 rows → C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/per_site_csvs\scats_970__high_street_rd_e_of_warrigal_rd.csv
Saved 2976 rows → C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/per_site_csvs\scats_970__high_street_rd_w_of_warrigal_rd.csv
Saved 2976 rows → C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/per_site_csvs\scats_970__warrigal_rd_n_of_high_street_rd.csv
Saved 2976 rows → C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/per_site_csvs\scats_970__warrigal_rd_s_of_high_street_rd.csv
Saved 2976 rows → C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Processed Data/per_site_csvs\scats_2000__burwood_hwy_e_of_warrigal_rd.csv
Saved 2976 rows → C:/Swinburne/2025Sem1/COS30019-Introduction to Artificial Intelligence/Assignment 2B/Proc